# EGAS00001004809-RLsearching

In [ ]:
import rpy2.robjects as robjects
import scanpy as sc
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
data_dir = '/home/qiuaodon/Desktop/'

In [ ]:
# read in the DEG results
DEG_T = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_T_cell.xlsx')
DEG_B = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_B_cell.xlsx')
DEG_M = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_M_cell.xlsx')
DEG_Epi = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_Epi_cell.xlsx')
DEG_Endo = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_Endo_cell.xlsx')
DEG_Fibro = pd.read_excel(data_dir + 'DEG-wholecelltype/EGAS00001004809_whole_Fibro_cell.xlsx')

In [ ]:
adata_T = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_T_cells.h5ad')
adata_B = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_B_cells.h5ad')
adata_M = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_M_cells.h5ad')
adata_Epi = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_Epi_cells.h5ad')
adata_Endo = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_Endo_cells.h5ad')
adata_Fibro = sc.read(data_dir + 'project_data_new/1863-counts_cells_cohort1_Fibro_cells.h5ad')

In [ ]:
sc.pl.umap(adata_M, color = 'CD40')

In [ ]:
sc.pl.umap(adata_B, color = 'CD40')

In [ ]:
sc.pl.umap(adata_T, color = 'CD40LG')

In [ ]:
adata = sc.read(data_dir + 'project_data_new/whole_cohort1_processed.h5ad')

In [ ]:
sc.pl.umap(adata, color = 'CD40')

In [ ]:
sc.pl.umap(adata, color = 'CD40LG')

In [ ]:
pseudo_T = scRNA2PseudoBulkAnnData(adata_T, sample_id_col='sample_id')
pseudo_B = scRNA2PseudoBulkAnnData(adata_B, sample_id_col='sample_id')
pseudo_M = scRNA2PseudoBulkAnnData(adata_M, sample_id_col='sample_id')
pseudo_Epi = scRNA2PseudoBulkAnnData(adata_Epi, sample_id_col='sample_id')
pseudo_Endo = scRNA2PseudoBulkAnnData(adata_Endo, sample_id_col='sample_id')
pseudo_Fibro = scRNA2PseudoBulkAnnData(adata_Fibro, sample_id_col='sample_id')

In [ ]:
pseudo_B

In [ ]:
# remove patient_id BIOKEY_22 and BIOKEY_28 from the pseudo bulk data
pseudo_T = pseudo_T[~pseudo_T.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]
pseudo_B = pseudo_B[~pseudo_B.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]
pseudo_M = pseudo_M[~pseudo_M.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]
pseudo_Epi = pseudo_Epi[~pseudo_Epi.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]
pseudo_Endo = pseudo_Endo[~pseudo_Endo.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]
pseudo_Fibro = pseudo_Fibro[~pseudo_Fibro.obs['patient_id'].isin(['BIOKEY_22', 'BIOKEY_28'])]

In [ ]:
pseudo_B.obs['patient_id'].unique()

In [ ]:
DEG_T

In [ ]:
# get the DEG names from the first column of the DEG results
DEG_T_names = DEG_T.iloc[:,0].values
DEG_B_names = DEG_B.iloc[:,0].values
DEG_M_names = DEG_M.iloc[:,0].values
DEG_Epi_names = DEG_Epi.iloc[:,0].values
DEG_Endo_names = DEG_Endo.iloc[:,0].values
DEG_Fibro_names = DEG_Fibro.iloc[:,0].values


In [ ]:
# draw the veen diagram for the varnames of pseudo_X and DEG_X
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
venn2([set(pseudo_Endo.var_names), set(DEG_Endo_names)], set_labels = ('pseudo', 'DEG'))
plt.show()

In [ ]:
# get the common genes between the pseudo and DEG of endo
common_genes_endo = list(set(pseudo_Endo.var_names) & set(DEG_Endo_names))

In [ ]:
venn2([set(pseudo_Fibro.var_names), set(DEG_Fibro_names)], set_labels = ('pseudo', 'DEG'))
plt.show()

In [ ]:
# extract the DEG expression from the pseudo-bulk data
DEG_T_exp = pseudo_T[:, DEG_T_names].X
DEG_T_exp = pd.DataFrame(DEG_T_exp, columns=DEG_T_names, index=pseudo_T.obs['sample_id'])
DEG_B_exp = pseudo_B[:, DEG_B_names].X
DEG_B_exp = pd.DataFrame(DEG_B_exp, columns=DEG_B_names, index=pseudo_B.obs['sample_id'])
DEG_M_exp = pseudo_M[:, DEG_M_names].X
DEG_M_exp = pd.DataFrame(DEG_M_exp, columns=DEG_M_names, index=pseudo_M.obs['sample_id'])
DEG_Epi_exp = pseudo_Epi[:, DEG_Epi_names].X
DEG_Epi_exp = pd.DataFrame(DEG_Epi_exp, columns=DEG_Epi_names, index=pseudo_Epi.obs['sample_id'])
# some of the DEG genes of endo are not in the pseudo-bulk data
DEG_Endo_exp = pseudo_Endo[:, common_genes_endo].X
DEG_Endo_exp = pd.DataFrame(DEG_Endo_exp, columns=common_genes_endo, index=pseudo_Endo.obs['sample_id'])
DEG_Fibro_exp = pseudo_Fibro[:, DEG_Fibro_names].X
DEG_Fibro_exp = pd.DataFrame(DEG_Fibro_exp, columns=DEG_Fibro_names, index=pseudo_Fibro.obs['sample_id'])

In [ ]:
cell_types = {
    'T': {'pseudo': pseudo_T, 'DEG_names': DEG_T_names},
    'B': {'pseudo': pseudo_B, 'DEG_names': DEG_B_names},
    'M': {'pseudo': pseudo_M, 'DEG_names': DEG_M_names},
    'Epi': {'pseudo': pseudo_Epi, 'DEG_names': DEG_Epi_names},
    'Endo': {'pseudo': pseudo_Endo, 'DEG_names': common_genes_endo},
    'Fibro': {'pseudo': pseudo_Fibro, 'DEG_names': DEG_Fibro_names}
}
DEG_exp_dfs = {}
for cell_type, info in cell_types.items():
    DEG_names = [name + f'_{cell_type}' for name in info['DEG_names']]
    DEG_exp = info['pseudo'][:, info['DEG_names']].X
    if hasattr(DEG_exp, "toarray"):
        DEG_exp = DEG_exp.toarray()
    
    DEG_exp_df = pd.DataFrame(DEG_exp, columns=DEG_names, index=info['pseudo'].obs['sample_id'])
    
    DEG_exp_dfs[cell_type] = DEG_exp_df

In [ ]:
# merge the DEG expression data
DEG_exp = pd.concat(DEG_exp_dfs.values(), axis=1)
# remove the sample_id with NA
DEG_exp = DEG_exp.dropna()
# add one column for the treatment, 0 for sample with 'Pre' and 1 for sample with 'On'
DEG_exp['treatment'] = DEG_exp.index.str.contains('On').astype(int)
DEG_exp

In [ ]:
# save the DEG expression data
DEG_exp.to_csv(data_dir + 'DEG-wholecelltype/DEG_exp.csv')


In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()
# convert the DEG expression to R
DEG_exp_r = pandas2ri.py2rpy(DEG_exp)
robjects.r.assign('DEG_exp', DEG_exp_r)

# run the IV regression in R
r_string = """
DEG_exp

"""

robjects.r(r_string)

# load in the LR pairs

In [ ]:
lrpair = pd.read_csv(data_dir + 'project_data_new/lr_network_unique.tsv', sep='\t')

In [ ]:
lrpair

In [ ]:
# get the lr pairs including ligand(from) and receptor(to) from the lrpair file with database as 'kegg'
lrpair_kegg = lrpair[lrpair['database'] == 'kegg']
lrpair_kegg = lrpair_kegg[['from', 'to']]
# change from as L and to as R
lrpair_kegg.columns = ['L', 'R']
lrpair_kegg

In [ ]:
# get the L(from) value in the pseudo_T only if the L can be found in pseudo_T
valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_T.var_names]
L_T = pseudo_T[:, valid_columns].X
L_T = pd.DataFrame(L_T, columns=valid_columns, index=pseudo_T.obs['sample_id'])
L_T = L_T.dropna()

# drop the columns with the same name
L_T = L_T.loc[:,~L_T.columns.duplicated()]
L_T


In [ ]:
# get the R(to) values in the pseudo_B, M, Epi, Endo, Fibro only if the R can be found in the corresponding pseudo-bulk data
valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_B.var_names]
R_B = pseudo_B[:, valid_columns].X
R_B = pd.DataFrame(R_B, columns=valid_columns, index=pseudo_B.obs['sample_id'])
R_B = R_B.dropna()
R_B = R_B.loc[:,~R_B.columns.duplicated()]

valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_M.var_names]
R_M = pseudo_M[:, valid_columns].X
R_M = pd.DataFrame(R_M, columns=valid_columns, index=pseudo_M.obs['sample_id'])
R_M = R_M.dropna()
R_M = R_M.loc[:,~R_M.columns.duplicated()]

valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_Epi.var_names]
R_Epi = pseudo_Epi[:, valid_columns].X
R_Epi = pd.DataFrame(R_Epi, columns=valid_columns, index=pseudo_Epi.obs['sample_id'])
R_Epi = R_Epi.dropna()
R_Epi = R_Epi.loc[:,~R_Epi.columns.duplicated()]

valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_Endo.var_names]
R_Endo = pseudo_Endo[:, valid_columns].X
R_Endo = pd.DataFrame(R_Endo, columns=valid_columns, index=pseudo_Endo.obs['sample_id'])
R_Endo = R_Endo.dropna()
R_Endo = R_Endo.loc[:,~R_Endo.columns.duplicated()]

valid_columns = [col for col in lrpair_kegg['L'] if col in pseudo_Fibro.var_names]
R_Fibro = pseudo_Fibro[:, valid_columns].X
R_Fibro = pd.DataFrame(R_Fibro, columns=valid_columns, index=pseudo_Fibro.obs['sample_id'])
R_Fibro = R_Fibro.dropna()
R_Fibro = R_Fibro.loc[:,~R_Fibro.columns.duplicated()]


# find the significant pairs 

### test

In [ ]:
# read in DEG pairs
DEG_pairs = pd.read_csv(data_dir + 'project_data_new/gene_pairs.csv')

In [ ]:
DEG_pairs

In [ ]:
# only change Gene2 which is TNKS2.AS1_B to TNKS2-AS1_B
DEG_pairs['Gene2'] = DEG_pairs['Gene2'].str.replace('TNKS2.AS1_B', 'TNKS2-AS1_B')



### load in the IVtest pairs

In [ ]:
# read in DEG_B pairs
DEG_B_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/data_IVtest_result/B_significant_gene_pairs.csv')
DEG_M_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/data_IVtest_result/M_significant_gene_pairs.csv')
DEG_Epi_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/data_IVtest_result/Epi_significant_gene_pairs.csv')
DEG_Endo_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/data_IVtest_result/Endo_significant_gene_pairs.csv')
DEG_Fibro_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/data_IVtest_result/Fibro_significant_gene_pairs.csv')

In [ ]:
DEG_B_pairs

In [ ]:
DEG_B_pairs = DEG_B_pairs[DEG_B_pairs['p_value'] < 0.001]


In [ ]:
# get the Gene1 and Gene2 from the gene_pair column, they are separated by '_vs_'
DEG_B_pairs['Gene1'] = DEG_B_pairs['gene_pair'].str.split('_vs_').str[0]
DEG_B_pairs['Gene2'] = DEG_B_pairs['gene_pair'].str.split('_vs_').str[1]
DEG_M_pairs['Gene1'] = DEG_M_pairs['gene_pair'].str.split('_vs_').str[0]
DEG_M_pairs['Gene2'] = DEG_M_pairs['gene_pair'].str.split('_vs_').str[1]
DEG_Epi_pairs['Gene1'] = DEG_Epi_pairs['gene_pair'].str.split('_vs_').str[0]
DEG_Epi_pairs['Gene2'] = DEG_Epi_pairs['gene_pair'].str.split('_vs_').str[1]
DEG_Endo_pairs['Gene1'] = DEG_Endo_pairs['gene_pair'].str.split('_vs_').str[0]
DEG_Endo_pairs['Gene2'] = DEG_Endo_pairs['gene_pair'].str.split('_vs_').str[1]
DEG_Fibro_pairs['Gene1'] = DEG_Fibro_pairs['gene_pair'].str.split('_vs_').str[0]
DEG_Fibro_pairs['Gene2'] = DEG_Fibro_pairs['gene_pair'].str.split('_vs_').str[1]
# get the Gene1 and Gene2 from the DEG_pairs
DEG_B_pairs = DEG_B_pairs[['Gene1', 'Gene2']]
DEG_M_pairs = DEG_M_pairs[['Gene1', 'Gene2']]
DEG_Epi_pairs = DEG_Epi_pairs[['Gene1', 'Gene2']]
DEG_Endo_pairs = DEG_Endo_pairs[['Gene1', 'Gene2']]
DEG_Fibro_pairs = DEG_Fibro_pairs[['Gene1', 'Gene2']]


In [ ]:
DEG_M_pairs['Gene2'].nunique()

## find the significant pairs between T and B

In [ ]:
from causallearn.utils.cit import CIT

# Initialize a dataframe to save the results
results_B = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])

# Iterate through each pair of DEGs
for g1, g2 in DEG_B_pairs.values:
    # Check if g2 ends with '_B'
    if g2.endswith('_B'):
        # Iterate through each ligand-receptor pair
        for L, R in lrpair_kegg.values:
            # Check if L and R are in the DEG expression data
            if L in pseudo_T.var_names and R in pseudo_B.var_names:
                # Extract and clean the expression data for L and R
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id']).dropna()
                R_B = pseudo_B[:, R].X
                R_B = pd.DataFrame(R_B, columns=[R], index=pseudo_B.obs['sample_id']).dropna()

                # Rename columns to ensure they match for multiplication
                L_T.columns = [R + '_vs_' + L]
                R_B.columns = L_T.columns

                # Perform the multiplication and flatten to 1-dimensional
                L_R = L_T.multiply(R_B, axis=0).iloc[:, 0]

                # Check if g1 and g2 are in the DEG expression data
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    # Replace '.' with '-' if necessary
                    g1 = g1.replace('.', '-')
                    g2 = g2.replace('.', '-')
                    if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                        continue

                # Extract the expression data for g1 and g2
                g1_T = DEG_exp[g1]
                g2_B = DEG_exp[g2]

                # Create a dataframe with g1, g2, and L*R
                df = pd.DataFrame({g1: g1_T, g2: g2_B, 'L*R': L_R}).dropna()

                # Convert the dataframe to a numpy array
                df_numpy = df.to_numpy()

                # Perform the Fisher-Z test
                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj(0, 1, [2])

                # Save results if the p-value is greater than 0.1
                if pValue > 0.1:
                    new_row = {'g1': g1, 'g2': g2, 'L1': L, 'R1': R,'L2': L2, 'R2': R2, 'pValue': pValue}
                    results_B = pd.concat([results_B, pd.DataFrame(new_row, index=[0])], ignore_index=True)

# Print the results
print(results_B)


In [ ]:
from causallearn.utils.cit import CIT
# make a dataframe prepared to save the results
results_B = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])

for g1, g2 in DEG_B_pairs.values:
    # check if g2 is ended with '_B'
    if g2.endswith('_B'):

        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in pseudo_T.var_names and R  in pseudo_B.var_names:
                # get the L_T and R_B from pseudo_T and pseudo_B
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
                L_T = L_T.dropna()
                R_B = pseudo_B[:, R].X
                R_B = pd.DataFrame(R_B, columns=[R], index=pseudo_B.obs['sample_id'])
                R_B = R_B.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_B.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_B, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    # change '.' to '_'
                    g1 = g1.replace('.', '-')
                    g2 = g2.replace('.', '-')
                    if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                        continue
                
                g1_T = DEG_exp[g1]
                g2_B = DEG_exp[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_B, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz") # construct a CIT instance with data and method name
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.1:
                    # save the g1, g2, L, R, pValue to a dataframe
                    results_B = pd.concat([results_B, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)
                    
print(results_B)

In [ ]:
gene1 = results_B['g1'].unique()
# convert gene1 to a string and remove the _T in the gene1
gene1 = str(gene1).replace('_T', '')
print(gene1)

gene2 = results_B['g2'].unique()
# convert gene2 to a string and remove the _B in the gene2
gene2 = str(gene2).replace('_B', '')
print(gene2)

In [ ]:
# add columns for correlation between g1 and g2
results_B['correlation(T vs B)'] = np.nan
results_B['correlation(T vs B)'] = results_B['correlation(T vs B)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and g2 from pseudo_B
for i, row in results_B.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    g1_T = DEG_exp[g1]
    g2_B = DEG_exp[g2]  
    corr = g1_T.corr(g2_B)
    results_B.at[i, 'correlation(T vs B)'] = corr


In [ ]:

# add columns for correlation between g1 and L
results_B['correlation(T vs L)'] = np.nan
results_B['correlation(T vs L)'] = results_B['correlation(T vs L)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and L from pseudo_T
for i, row in results_B.iterrows():
    g1 = row['g1']
    L = row['L']
    g1_T = DEG_exp[g1]
    L_T = pd.DataFrame(pseudo_T[:, L].X, columns=[L], index=pseudo_T.obs['sample_id']) 
    # Convert ArrayView to DataFrame
    L_T.columns = [L]  # Ensure column names match
    corr = g1_T.corr(L_T.squeeze())  # Use squeeze() to convert 1-dimensional DataFrame to Series
    results_B.at[i, 'correlation(T vs L)'] = corr
    if g1 == 'PMAIP1_T':
        print(g1_T)


In [ ]:

# add columns for correlation between R and g2
results_B['correlation(R vs B)'] = np.nan
results_B['correlation(R vs B)'] = results_B['correlation(R vs B)'].astype('object')
# add in the values for correlation, R from pseudo_B and g2 from pseudo_B
for i, row in results_B.iterrows():
    g2 = row['g2']
    R = row['R']
    g2_B = DEG_exp[g2]
    R_B = pd.DataFrame(pseudo_B[:, R].X, columns=[R], index=pseudo_B.obs['sample_id'])
    # Convert ArrayView to DataFrame
    R_B.columns = [R]  # Ensure column names match
    corr = R_B.squeeze().corr(g2_B)
    results_B.at[i, 'correlation(R vs B)'] = corr

In [ ]:

# add columns for correlation between L*R and g2
results_B['correlation(L*R vs B)'] = np.nan
results_B['correlation(L*R vs B)'] = results_B['correlation(L*R vs B)'].astype('object')
# add in the values for correlation, L*R from pseudo_T and g2 from pseudo_B
for i, row in results_B.iterrows():
    g2 = row['g2']
    g2_B = DEG_exp[g2]
    L_T = pd.DataFrame(pseudo_T[:, row['L']].X, columns=[row['L']], index=pseudo_T.obs['sample_id'])
    R_B = pd.DataFrame(pseudo_B[:, row['R']].X, columns=[row['R']], index=pseudo_B.obs['sample_id'])
    # get L*R from L_T and R_B based on sample_id
    # make sure the column names match
    L_T.columns = [row['R']+'_vs_'+row['L']]
    R_B.columns = L_T.columns
    g2_B.columns = L_T.columns
    L_R = L_T.multiply(R_B, axis=0)
    corr = L_R.squeeze().corr(g2_B)
    results_B.at[i, 'correlation(L*R vs B)'] = corr


In [ ]:
results_B

### example

In [ ]:
g1 = 'TSC22D3_T'
g2 = 'TNKS_B'
g1_T = DEG_exp[g1]
g2_B = DEG_exp[g2]
df = pd.DataFrame({g1: g1_T, g2: g2_B})

In [ ]:
from causallearn.utils.cit import CIT

L = 'CXCL1'
R = 'CXCR3'
L_T = pseudo_T[:, L].X
L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
L_T = L_T.dropna()
R_B = pseudo_B[:, R].X
R_B = pd.DataFrame(R_B, columns=[R], index=pseudo_B.obs['sample_id'])
R_B = R_B.dropna()

# Ensure the column names match
L_T.columns = [R+'_vs_'+L]
R_B.columns = L_T.columns

# perform the multiplication and change L_R to 1-dimensional
L_R = L_T.multiply(R_B, axis=0)
L_R = L_R.iloc[:,0]

# make a dataframe g1, g2 and L*R
g1_T = DEG_exp[g1]
g2_B = DEG_exp[g2]
df = pd.DataFrame({g1: g1_T, g2: g2_B, 'L*R': L_R})
# remove the rows with NA
df = df.dropna()

# convert df to numpy array
df_numpy = df.to_numpy()

fisherz_obj = CIT(df_numpy, "fisherz") # construct a CIT instance with data and method name
pValue = fisherz_obj( 0, 1, [2] )
pValue

In [ ]:
df 


In [ ]:
#find the R genes that is in the pseudo_B.var_names
valid_columns = [col for col in lrpair_kegg['R'] if col in pseudo_B.var_names]
valid_columns

In [ ]:
L_R = L_T.multiply(R_B, axis=0)

### plot figures: G_B vs G_T, G_B res vs G_T, L vs G_T, G_B vs R

In [ ]:
# plot the figure of all g1 and g2 scatter plot and also add the treatment as color
import seaborn as sns
import matplotlib.pyplot as plt
# all g1 and g2 in result_B scatter plot and also add the treatment as color
for g1, g2 in results_B[['g1', 'g2']].values:
    g1_T = DEG_exp[g1]
    g2_B = DEG_exp[g2]
    df = pd.DataFrame({g1: g1_T, g2: g2_B, 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=g1, y=g2, hue='treatment')
    # add the correalation coefficient
    corr = df[g1].corr(df[g2])
    plt.title(f'g1: {g1}, g2: {g2}, corr: {corr}')
    plt.show()



In [ ]:
# plot the figure of all R and g2 scatter plot and also add the treatment as color
for R, g2 in results_B[['R', 'g2']].values:
    R_B = pseudo_B[:, R].X
    R_B = pd.DataFrame(R_B, columns=[R], index=pseudo_B.obs['sample_id'])
    R_B = R_B.dropna()
    g2_B = DEG_exp[g2]
    df = pd.DataFrame({R: R_B.iloc[:,0], g2: g2_B, 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=R, y=g2, hue='treatment')
    plt.show()

In [ ]:
# plot the figure of all g1 and L scatter plot and also add the treatment as color
for g1, L in results_B[['g1', 'L']].values:
    g1_T = DEG_exp[g1]
    L_T = pseudo_T[:, L].X
    L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
    L_T = L_T.dropna()
    df = pd.DataFrame({g1: g1_T, L: L_T.iloc[:,0], 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=g1, y=L, hue='treatment')
    plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
#add one column for correlation between residuals and g1
results_B['correlation(residuals vs g1)'] = np.nan
results_B['correlation(residuals vs g1)'] = results_B['correlation(residuals vs g1)'].astype('object')
for index, row in results_B.iterrows():
    L = row['L']
    R = row['R']

    # Calculate L*R
    L_values = pseudo_T[:, L].X
    R_values = pseudo_B[:, R].X
    L_R = L_values * R_values

    # Prepare the data for regression
    X = sm.add_constant(L_R)  # add a constant term for the intercept
    DEG_B = row['g2']  # DEG_B column name from results_B
    Y_B = DEG_exp[DEG_B]

    # Regress DEG_B on L*R to get residuals
    model_B = sm.OLS(Y_B, X).fit()
    residuals_B = model_B.resid
    # add correlation between residuals and g1
    corr = residuals_B.corr(DEG_exp[row['g1']])
    # add the correlation to the results_B
    results_B.at[index, 'correlation(residuals vs g1)'] = corr
    # show color for the treatment
    treatment = DEG_exp['treatment']
    # show which color is pre and after treatment
    treatment = treatment.map({0: 'blue', 1: 'orange'})
    # Plot the residuals against DEG_T
    plt.scatter(DEG_exp[row['g1']], residuals_B, c=treatment)
    plt.title(f'{row["g1"]} vs residuals of {row["g2"]}, corr: {corr}')
    plt.xlabel(row['g1'])
    plt.ylabel(f'Residuals of {row["g2"]}')
    plt.show()
    


In [ ]:
# save the results_B to excel
with pd.ExcelWriter(data_dir + 'project_data_new/data_IVtest_result/CIT_test_results_B.xlsx') as writer:
    results_B.to_excel(writer, sheet_name='B')

## find the significant pairs between T and M

In [ ]:
from causallearn.utils.cit import CIT
results_M = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])
for g1, g2 in DEG_M_pairs.values:
    # check if g2 is ended with '_M'
    if g2.endswith('_M'):
        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in pseudo_T.var_names and R  in pseudo_M.var_names:
                # get the L_T and R_B from pseudo_T and pseudo_M
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
                L_T = L_T.dropna()
                R_M = pseudo_M[:, R].X
                R_M = pd.DataFrame(R_M, columns=[R], index=pseudo_M.obs['sample_id'])
                R_M = R_M.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_M.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_M, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    continue
                
                g1_T = DEG_exp[g1]
                g2_M = DEG_exp[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_M, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.1:
                    results_M = pd.concat([results_M, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)
                
print(results_M)

In [ ]:
gene1 = results_M['g1'].unique()
# convert gene1 to a string and remove the _T in the gene1
gene1 = str(gene1).replace('_T', '')
print(gene1)

gene2 = results_M['g2'].unique()
# convert gene2 to a string and remove the _B in the gene2
gene2 = str(gene2).replace('_M', '')
print(gene2)

In [ ]:
# add columns for correlation between

# add columns for correlation between g1 and g2
results_M['correlation(T vs M)'] = np.nan
results_M['correlation(T vs M)'] = results_M['correlation(T vs M)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and g2 from pseudo_M
for i, row in results_M.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    g1_T = DEG_exp[g1]
    g2_M = DEG_exp[g2]  
    corr = g1_T.corr(g2_M)
    results_M.at[i, 'correlation(T vs M)'] = corr

# add columns for correlation between g1 and L
results_M['correlation(T vs L)'] = np.nan
results_M['correlation(T vs L)'] = results_M['correlation(T vs L)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and L from pseudo_T
for i, row in results_M.iterrows():
    g1 = row['g1']
    L = row['L']
    g1_T = DEG_exp[g1]
    L_T = pd.DataFrame(pseudo_T[:, L].X, columns=[L], index=pseudo_T.obs['sample_id']) 
    # Convert ArrayView to DataFrame
    L_T.columns = [L]  # Ensure column names match
    corr = g1_T.corr(L_T.squeeze())  # Use squeeze() to convert 1-dimensional DataFrame to Series
    results_M.at[i, 'correlation(T vs L)'] = corr

# add columns for correlation between R and g2
results_M['correlation(R vs M)'] = np.nan
results_M['correlation(R vs M)'] = results_M['correlation(R vs M)'].astype('object')
# add in the values for correlation, R from pseudo_M and g2 from pseudo_M
for i, row in results_M.iterrows():
    g2 = row['g2']
    R = row['R']
    g2_M = DEG_exp[g2]
    R_M = pd.DataFrame(pseudo_M[:, R].X, columns=[R], index=pseudo_M.obs['sample_id'])
    # Convert ArrayView to DataFrame
    R_M.columns = [R]  # Ensure column names match
    corr = R_M.squeeze().corr(g2_M)
    results_M.at[i, 'correlation(R vs M)'] = corr

# add columns for correlation between L*R and g2
results_M['correlation(L*R vs M)'] = np.nan
results_M['correlation(L*R vs M)'] = results_M['correlation(L*R vs M)'].astype('object')
# add in the values for correlation, L*R from pseudo_T and g2 from pseudo_M
for i, row in results_M.iterrows():
    g2 = row['g2']
    g2_M = DEG_exp[g2]
    L_T = pd.DataFrame(pseudo_T[:, row['L']].X, columns=[row['L']], index=pseudo_T.obs['sample_id'])
    R_M = pd.DataFrame(pseudo_M[:, row['R']].X, columns=[row['R']], index=pseudo_M.obs['sample_id'])
    # get L*R from L_T and R_M based on sample_id
    # make sure the column names match
    L_T.columns = [row['R']+'_vs_'+row['L']]
    R_M.columns = L_T.columns
    g2_M.columns = L_T.columns
    L_R = L_T.multiply(R_M, axis=0)
    corr = L_R.squeeze().corr(g2_M)
    results_M.at[i, 'correlation(L*R vs M)'] = corr


In [ ]:
results_M

In [ ]:
# save the results_M to excel
with pd.ExcelWriter(data_dir + 'project_data_new/data_IVtest_result/CIT_test_results_M.xlsx') as writer:
    results_M.to_excel(writer, sheet_name='M')

### plot figures

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# plot the figure of all g1 and g2 scatter plot and also add the treatment as color
for g1, g2 in results_M[['g1', 'g2']].values:
    g1_T = DEG_exp[g1]
    g2_M = DEG_exp[g2]
    df = pd.DataFrame({g1: g1_T, g2: g2_M, 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=g1, y=g2, hue='treatment')
    # add the correalation coefficient
    corr = df[g1].corr(df[g2])
    plt.title(f'g1: {g1}, g2: {g2}, corr: {corr}')
    plt.show()

In [ ]:
for R, g2 in results_M[['R', 'g2']].values:
    R_B = pseudo_M[:, R].X
    R_B = pd.DataFrame(R_B, columns=[R], index=pseudo_M.obs['sample_id'])
    R_B = R_B.dropna()
    g2_B = DEG_exp[g2]
    df = pd.DataFrame({R: R_B.iloc[:,0], g2: g2_M, 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=R, y=g2, hue='treatment')
    plt.show()

In [ ]:
# plot the figure of all g1 and L scatter plot and also add the treatment as color
for g1, L in results_M[['g1', 'L']].values:
    g1_T = DEG_exp[g1]
    L_T = pseudo_T[:, L].X
    L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
    L_T = L_T.dropna()
    df = pd.DataFrame({g1: g1_T, L: L_T.iloc[:,0], 'treatment': DEG_exp['treatment']})
    df = df.dropna()
    sns.scatterplot(data=df, x=g1, y=L, hue='treatment')
    plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt 
for index, row in results_M.iterrows():
    L = row['L']
    R = row['R']

    # Calculate L*R
    L_values = pseudo_T[:, L].X
    R_values = pseudo_M[:, R].X
    L_R = L_values * R_values

    # Prepare the data for regression
    X = sm.add_constant(L_R)  # add a constant term for the intercept
    DEG_M = row['g2']  # DEG_B column name from results_B
    Y_M = DEG_exp[DEG_M]

    # Regress DEG_B on L*R to get residuals
    model_M = sm.OLS(Y_M, X).fit()
    residuals_M = model_M.resid
    # add correlation between residuals and g1
    corr = residuals_M.corr(DEG_exp[row['g1']])
    results_M.at[index, 'correlation(residuals vs g1)'] = corr
    # show color for the treatment
    treatment = DEG_exp['treatment']
    # show which color is pre and after treatment
    treatment = treatment.map({0: 'blue', 1: 'orange'})
    # Plot the residuals against DEG_T
    plt.scatter(DEG_exp[row['g1']], residuals_M, c=treatment)
    plt.title(f'{row["g1"]} vs residuals of {row["g2"]}, corr: {corr}')
    plt.xlabel(row['g1'])
    plt.ylabel(f'Residuals of {row["g2"]}')
    plt.show()

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt 
for index, row in results_M.iterrows():
    L = row['L']
    R = row['R']

    # Calculate L*R
    L_values = pseudo_T[:, L].X
    R_values = pseudo_M[:, R].X
    L_R = L_values * R_values

    # Prepare the data for regression
    X = sm.add_constant(L_R)  # add a constant term for the intercept
    DEG_M = row['g2']  # DEG_B column name from results_B
    Y_M = DEG_exp[DEG_M]

    # Regress DEG_B on L*R to get residuals
    model_M = sm.OLS(Y_M, X).fit()
    residuals_M = model_M.resid
    # add correlation between residuals and g1
    corr = residuals_M.corr(DEG_exp[row['g1']])
    results_M.at[index, 'correlation(residuals vs g1)'] = corr


## find the significant pairs between T and Endo

In [ ]:
# find the significant pairs between T and Endo
results_T_Endo = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])
for g1, g2 in DEG_Endo_pairs.values:
    # check if g2 is ended with '_Endo'
    if g2.endswith('_Endo'):
        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in pseudo_T.var_names and R  in pseudo_Endo.var_names:
                # get the L_T and R_Endo from pseudo_T and pseudo_Endo
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
                L_T = L_T.dropna()
                R_Endo = pseudo_Endo[:, R].X
                R_Endo = pd.DataFrame(R_Endo, columns=[R], index=pseudo_Endo.obs['sample_id'])
                R_Endo = R_Endo.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_Endo.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_Endo, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    continue
                
                g1_T = DEG_exp[g1]
                g2_Endo = DEG_exp[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_Endo, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.1:
                    # save the g1, g2, L, R, pValue to a dataframe
                    results_T_Endo = pd.concat([results_T_Endo, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)

print(results_T_Endo)

In [ ]:
# add columns for correlation between

# add columns for correlation between g1 and g2
results_T_Endo['correlation(T vs Endo)'] = np.nan
results_T_Endo['correlation(T vs Endo)'] = results_T_Endo['correlation(T vs Endo)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and g2 from pseudo_Endo
for i, row in results_T_Endo.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    g1_T = DEG_exp[g1]
    g2_Endo = DEG_exp[g2]  
    corr = g1_T.corr(g2_Endo)
    results_T_Endo.at[i, 'correlation(T vs Endo)'] = corr

# add columns for correlation between g1 and L
results_T_Endo['correlation(T vs L)'] = np.nan
results_T_Endo['correlation(T vs L)'] = results_T_Endo['correlation(T vs L)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and L from pseudo_T
for i, row in results_T_Endo.iterrows():
    g1 = row['g1']
    L = row['L']
    g1_T = DEG_exp[g1]
    L_T = pd.DataFrame(pseudo_T[:, L].X, columns=[L], index=pseudo_T.obs['sample_id']) 
    # Convert ArrayView to DataFrame
    L_T.columns = [L]  # Ensure column names match
    corr = g1_T.corr(L_T.squeeze())  # Use squeeze() to convert 1-dimensional DataFrame to Series
    results_T_Endo.at[i, 'correlation(T vs L)'] = corr

# add columns for correlation between R and g2
results_T_Endo['correlation(R vs Endo)'] = np.nan
results_T_Endo['correlation(R vs Endo)'] = results_T_Endo['correlation(R vs Endo)'].astype('object')
# add in the values for correlation, R from pseudo_Endo and g2 from pseudo_Endo
for i, row in results_T_Endo.iterrows():
    g2 = row['g2']
    R = row['R']
    g2_Endo = DEG_exp[g2]
    R_Endo = pd.DataFrame(pseudo_Endo[:, R].X, columns=[R], index=pseudo_Endo.obs['sample_id'])
    # Convert ArrayView to DataFrame
    R_Endo.columns = [R]  # Ensure column names match
    corr = R_Endo.squeeze().corr(g2_Endo)
    results_T_Endo.at[i, 'correlation(R vs Endo)'] = corr

# add columns for correlation between L*R and g2
results_T_Endo['correlation(L*R vs Endo)'] = np.nan
results_T_Endo['correlation(L*R vs Endo)'] = results_T_Endo['correlation(L*R vs Endo)'].astype('object')
# add in the values for correlation, L*R from pseudo_T and g2 from pseudo_Endo
for i, row in results_T_Endo.iterrows():
    g2 = row['g2']
    g2_Endo = DEG_exp[g2]
    L_T = pd.DataFrame(pseudo_T[:, row['L']].X, columns=[row['L']], index=pseudo_T.obs['sample_id'])
    R_Endo = pd.DataFrame(pseudo_Endo[:, row['R']].X, columns=[row['R']], index=pseudo_Endo.obs['sample_id'])
    # get L*R from L_T and R_Endo based on sample_id
    # make sure the column names match
    L_T.columns = [row['R']+'_vs_'+row['L']]
    R_Endo.columns = L_T.columns
    g2_Endo.columns = L_T.columns
    L_R = L_T.multiply(R_Endo, axis=0)
    corr = L_R.squeeze().corr(g2_Endo)
    results_T_Endo.at[i, 'correlation(L*R vs Endo)'] = corr

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt 
for index, row in results_T_Endo.iterrows():
    L = row['L']
    R = row['R']

    # Calculate L*R
    L_values = pseudo_T[:, L].X
    R_values = pseudo_Endo[:, R].X
    L_R = L_values * R_values

    # Prepare the data for regression
    X = sm.add_constant(L_R)  # add a constant term for the intercept
    DEG_Endo = row['g2']  # DEG_B column name from results_B
    Y_Endo = DEG_exp[DEG_Endo]

    # Regress DEG_B on L*R to get residuals
    model_Endo = sm.OLS(Y_Endo, X).fit()
    residuals_Endo = model_Endo.resid
    # add correlation between residuals and g1
    corr = residuals_Endo.corr(DEG_exp[row['g1']])
    results_T_Endo.at[index, 'correlation(residuals vs g1)'] = corr
  

In [ ]:
results_T_Endo

In [ ]:
# save the results_T_Endo to excel
with pd.ExcelWriter(data_dir + 'project_data_new/data_IVtest_result/CIT_test_results_Endo.xlsx') as writer:
    results_T_Endo.to_excel(writer, sheet_name='Endo')

## find the significant pairs between T and Epi

In [ ]:
# find the significant pairs between T and Epi
results_T_Epi = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])
for g1, g2 in DEG_Epi_pairs.values:
    # check if g2 is ended with '_Epi'
    if g2.endswith('_Epi'):
        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in pseudo_T.var_names and R in pseudo_Epi.var_names:
                # get the L_T and R_Epi from pseudo_T and pseudo_Epi
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
                L_T = L_T.dropna()
                R_Epi = pseudo_Epi[:, R].X
                R_Epi = pd.DataFrame(R_Epi, columns=[R], index=pseudo_Epi.obs['sample_id'])
                R_Epi = R_Epi.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_Epi.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_Epi, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    continue
                
                g1_T = DEG_exp[g1]
                g2_Epi = DEG_exp[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_Epi, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.01:
                    # save the g1, g2, L, R, pValue to a dataframe
                    results_T_Epi = pd.concat([results_T_Epi, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)

print(results_T_Epi)

## find the significant pairs between T and Fibro

In [ ]:
# find the significant pairs between T and Fibro
results_T_Fibro = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])
for g1, g2 in DEG_Fibro_pairs.values:
    # check if g2 is ended with '_Fibro'
    if g2.endswith('_Fibro'):
        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in pseudo_T.var_names and R in pseudo_Fibro.var_names:
                # get the L_T and R_Fibro from pseudo_T and pseudo_Fibro
                L_T = pseudo_T[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=pseudo_T.obs['sample_id'])
                L_T = L_T.dropna()
                R_Fibro = pseudo_Fibro[:, R].X
                R_Fibro = pd.DataFrame(R_Fibro, columns=[R], index=pseudo_Fibro.obs['sample_id'])
                R_Fibro = R_Fibro.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_Fibro.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_Fibro, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
                    # change the '.' to '-'
                    g1 = g1.replace('.', '-')
                    g2 = g2.replace('.', '-')
                
                g1_T = DEG_exp[g1]
                g2_Fibro = DEG_exp[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_Fibro, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.1:
                    # save the g1, g2, L, R, pValue to a dataframe
                    results_T_Fibro = pd.concat([results_T_Fibro, pd.DataFrame({'g1': [g1], ' g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)

print(results_T_Fibro)

In [ ]:
DEG_Fibro_pairs

In [ ]:
# add columns for correlation between

# add columns for correlation between g1 and g2
results_T_Fibro['correlation(T vs Fibro)'] = np.nan
results_T_Fibro['correlation(T vs Fibro)'] = results_T_Fibro['correlation(T vs Fibro)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and g2 from pseudo_Fibro
for i, row in results_T_Fibro.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    if g1 not in DEG_exp.columns or g2 not in DEG_exp.columns:
        continue
    g1_T = DEG_exp[g1]
    g2_Fibro = DEG_exp[g2]  
    corr = g1_T.corr(g2_Fibro)
    results_T_Fibro.at[i, 'correlation(T vs Fibro)'] = corr

# add columns for correlation between g1 and L
results_T_Fibro['correlation(T vs L)'] = np.nan
results_T_Fibro['correlation(T vs L)'] = results_T_Fibro['correlation(T vs L)'].astype('object')
# add in the values for correlation, g1 from pseudo_T and L from pseudo_T
for i, row in results_T_Fibro.iterrows():
    g1 = row['g1']
    L = row['L']
    if g1 not in DEG_exp.columns:
        continue
    g1_T = DEG_exp[g1]
    L_T = pd.DataFrame(pseudo_T[:, L].X, columns=[L], index=pseudo_T.obs['sample_id']) 
    # Convert ArrayView to DataFrame
    L_T.columns = [L]  # Ensure column names match
    corr = g1_T.corr(L_T.squeeze())  # Use squeeze() to convert 1-dimensional DataFrame to Series
    results_T_Fibro.at[i, 'correlation(T vs L)'] = corr

# add columns for correlation between R and g2
results_T_Fibro['correlation(R vs Fibro)'] = np.nan
results_T_Fibro['correlation(R vs Fibro)'] = results_T_Fibro['correlation(R vs Fibro)'].astype('object')
# add in the values for correlation, R from pseudo_Fibro and g2 from pseudo_Fibro
for i, row in results_T_Fibro.iterrows():
    g2 = row['g2']
    R = row['R']
    if g2 not in DEG_exp.columns:
        continue
    g2_Fibro = DEG_exp[g2]
    R_Fibro = pd.DataFrame(pseudo_Fibro[:, R].X, columns=[R], index=pseudo_Fibro.obs['sample_id'])
    # Convert ArrayView to DataFrame
    R_Fibro.columns = [R]  # Ensure column names match
    corr = R_Fibro.squeeze().corr(g2_Fibro)
    results_T_Fibro.at[i, 'correlation(R vs Fibro)'] = corr

# add columns for correlation between L*R and g2
results_T_Fibro['correlation(L*R vs Fibro)'] = np.nan
results_T_Fibro['correlation(L*R vs Fibro)'] = results_T_Fibro['correlation(L*R vs Fibro)'].astype('object')
# add in the values for correlation, L*R from pseudo_T and g2 from pseudo_Fibro
for i, row in results_T_Fibro.iterrows():
    if g2 not in DEG_exp.columns:
        continue
    g2 = row['g2']
    g2_Fibro = DEG_exp[g2]
    L_T = pd.DataFrame(pseudo_T[:, row['L']].X, columns=[row['L']], index=pseudo_T.obs['sample_id'])
    R_Fibro = pd.DataFrame(pseudo_Fibro[:, row['R']].X, columns=[row['R']], index=pseudo_Fibro.obs['sample_id'])
    # get L*R from L_T and R_Fibro based on sample_id
    # make sure the column names match
    L_T.columns = [row['R']+'_vs_'+row['L']]
    R_Fibro.columns = L_T.columns
    g2_Fibro.columns = L_T.columns
    L_R = L_T.multiply(R_Fibro, axis=0)
    corr = L_R.squeeze().corr(g2_Fibro)
    results_T_Fibro.at[i, 'correlation(L*R vs Fibro)'] = corr



In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt 
for index, row in results_T_Fibro.iterrows():
    L = row['L']
    R = row['R']

    # Calculate L*R
    L_values = pseudo_T[:, L].X
    R_values = pseudo_Fibro[:, R].X
    L_R = L_values * R_values

    # Prepare the data for regression
    X = sm.add_constant(L_R)  # add a constant term for the intercept
    DEG_Fibro = row['g2']  # DEG_B column name from results_B
    if DEG_Fibro not in DEG_exp.columns:
        continue
    Y_Fibro = DEG_exp[DEG_Fibro]

    # Regress DEG_B on L*R to get residuals
    model_Fibro = sm.OLS(Y_Fibro, X).fit()
    residuals_Fibro = model_Fibro.resid
    # add correlation between residuals and g1
    corr = residuals_Fibro.corr(DEG_exp[row['g1']])
    results_T_Fibro.at[index, 'correlation(residuals vs g1)'] = corr

In [ ]:
pseudo_Fibro

In [ ]:
with pd.ExcelWriter(data_dir + 'project_data_new/data_IVtest_result/CIT_test_results_Fibro.xlsx') as writer:
    results_T_Fibro.to_excel(writer, sheet_name='T_Fibro')

In [ ]:
# save the result of B, M, Endo together to excel
with pd.ExcelWriter(data_dir + 'project_data_new/data_IVtest_result/CIT_test_results.xlsx') as writer:
    results_B.to_excel(writer, sheet_name='B')
    results_M.to_excel(writer, sheet_name='M')
    results_T_Endo.to_excel(writer, sheet_name='T_Endo')
    results_T_Epi.to_excel(writer, sheet_name='T_Epi')
    results_T_Fibro.to_excel(writer, sheet_name='T_Fibro')

# visualize data with plots